# Projeto 2 - Ciência dos Dados

Rodrigo Sennati Mattar 

Lucca Nazari

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Lucca Nazari]***

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


O produto escolhido para ser o objeto de estudo no projeto foram os Airpods, fones de ouvido sem fio da Apple.
Como base de dados, importamos 750 para base de treinamento, em que nós classificamos os tweets e mais 250 de teste, para validar a eficácia do classificador.

In [5]:
#Produto escolhido:
produto = 'airpods'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 250

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [6]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
            break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(list(set(msgs)))

Salvando os dados em uma planilha Excel:

In [7]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem


Nós dividimos os tweets em 3  classificações diferentes (irrevelante,relevante positivo e relevante negativo). Ao classificar como irrevelante, nós estamos nos referindo a todos os tweets onde não há uma crítica explícita ao produto (aos que somente mencionam o produto). Em relação a segunda classificação, nós estamos nos referindo aos tweets onde houve uma crítica negativa explícita ao produto (como por exemplo os airpods acabam a bateria muito rapido, ou eu perdi meus airpods). A terceira classificação é para os tweets onde há uma crítica positiva explícita ao produto (como por exemplo "eu só queria um airpods")

___
### Montando o Classificador Naive-Bayes



Para realizar os códigos, é necessário importar as bibliotecas pd (pandas) e np (numpy).

In [8]:
import pandas as pd
import numpy as np

from IPython.display import display

pd.options.display.max_rows = 13 # Escolher quantos prints as funções do pandas fazem

Para a leitura dos tweets, é necessário "limpar" o tweet para que não exista influência de termos indesejados para a análise.

A primeira "limpeza" dos tweets foi realizada pela função cleanup, em que ela recebe uma string e remove todos os sinais de pontuação, vírgulas, pontos de interrogação e exclamação, dois-pontos e etc.

In [9]:
import re 

def cleanup(text):
    punctuation = '[!\-.:?;,]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

Em seguida, é necessário importar os tweets do excel, através do pandas.

In [10]:
tweets = pd.read_excel('airpods1.xlsx')

Importando as bibliotecas necessárias para o desenvolvimento do código.

In [11]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import os
from pandas import DataFrame
from numpy import arange
from numpy import percentile
import numpy as np
from matplotlib.font_manager import FontProperties
import string
import re 

Agora, já com os tweets importados e classificados, agrupamos todos os tweets em um único dataframe.
Após isso, separamos os tweets com base em sua classificação (1,2 ou 3), para facilitar os códigos.

In [12]:
# FILTRANDO TODOS OS TWEETS RELEVENATES
relev = tweets.loc[:,['Treinamento','Relevante']]
#FILTRANDO OS TWEETS DE ACORDO COM SUA RELEVANCIA (=1,=2,=3)
relev1 = tweets.loc[tweets.Relevante==1]
relev2 = tweets.loc[tweets.Relevante==2]
relev3 = tweets.loc[tweets.Relevante==3]
texto_relev1_ = relev1.Treinamento
texto_relev2_ = relev2.Treinamento
texto_relev3_ = relev3.Treinamento
texto_relev = relev.Treinamento

#SEPARANDO CADA GRUPO DE RELEVANCIA PARA UM ARQUIVO CSV
texto_relev1_.to_csv('relev1.csv', header=True, index=False, encoding='utf-8')
texto_relev2_.to_csv('relev2.csv', header=True, index=False, encoding='utf-8')
texto_relev3_.to_csv('relev3.csv', header=True, index=False, encoding='utf-8')
texto_relev.to_csv('relev.csv', header=True, index=False, encoding='utf-8')

Mesmo com a "limpeza" que foi realizada anteriormente, é necessário remover termos ilegíveis para evitar que o código dê erros.
Tal ação foi realizada pela função removetext, abaixo.

In [13]:
#ABRINDO OS ARQUIVOS CSV
#primeiro arquivo(relevancia =1)
df1 = pd.read_csv('relev1.csv')
df1 = df1.dropna(axis=0, how = 'any')
#LIMPANDO O TEXTO DE CADA ARQUIVO CSV E DANDO VALUE COUNTS 
def removetext(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

df1['Treinamento'] = df1['Treinamento'].apply(removetext)

#segund arquivo(relevancia = 2)
df2 = pd.read_csv('relev2.csv')
df2 = df2.dropna(axis=0, how = 'any')
df2['Treinamento'] = df2['Treinamento'].apply(removetext)


#terceiro arquivo (relevancia = 3)
df3 = pd.read_csv('relev3.csv')
df3 = df3.dropna(axis=0, how = 'any')
df3['Treinamento'] = df3['Treinamento'].apply(removetext)

#arquivo total
dft = pd.read_csv('relev.csv')
dft = dft.dropna(axis=0, how = 'any')
dft['Treinamento'] = dft['Treinamento'].apply(removetext)

Em seguida, foi desenvolvida a frequência relativa das palavras em relação a cada grupo de relevância, classificados como 1,2,3 e um grupo que contém todos os tweet analisados.

In [14]:
#CRIANDO A TABELA DE FREQUENCIA RELETIVA PARA CADA GRUPO DE RELEVANCIA DE UM TWEET
relev1_array = df1['Treinamento'].str.split(' ', expand=True).stack().value_counts()
relev2_array = df2['Treinamento'].str.split(' ', expand=True).stack().value_counts()
relev3_array = df3['Treinamento'].str.split(' ', expand=True).stack().value_counts()
relev_array = dft['Treinamento'].str.split(' ',expand = True).stack().value_counts()

No entanto, antes de classificar os tweets, é necessário separar os tweets em palavras isoladas, que possam ser analisadas individualmente. Assim, as palavras pertencentes a um tweet qualquer, são comparadas com à base de dados, dados do excel já classificados, para que possa ser atribuído a um determinado grupo relevante (1,2 ou 3).

Para o caso de uma nova palavra, ou seja, uma palavra que não tenha aparecido na base de dados, a função irá dividir 1 pela soma de todas as palavras no determinado grupo em que a palavra não está presente com todas as palavras presentes no dataframe. Isso foi feito justamente para que a frequência dessa palavra seja muito pequena, visto que por não ter aparecido anteriormente, não terá um peso relevante na análise.

Agora, para o caso de a palavra já presente na base de dados, a probabilidade será a divisão da frequência relativa da palavra em seu grupo relativo específico pela soma de todas as palavras do grupo específico com todas as palavras analisadas.

Após isso, o algoritmo compara as probabilidades encontradas de o tweet, soma de palavras, estar em cada grupo específico.
Como resposta, a função retorna a maior probabilidade entre os trÊs grupos relevantes. Logo, o tweet terá sido classificado como mais provável de ser um tweet irrelevante (1), tweet relevante com crítica negativa (2) ou tweet relevante com crítica positiva (3).

In [24]:
#PROBABILIDADE DE UMA PALAVRA EM UM TWEET ALEATORIO (VERIFICAR REPETIDAS)
P1 = 1
P2 = 1 
P3 = 1

def robo(tweet_teste,P1,P2,P3):
    tweet_teste = str(tweet_teste)
    tweet_teste = tweet_teste.lower()
    tweet_teste = cleanup(tweet_teste)
    tweet_teste = removetext(tweet_teste)
    tweet_teste = pd.Series(tweet_teste.split())
    lista_tweet = list(tweet_teste)
    for palavra in lista_tweet:
        if palavra not in relev1_array:
            P1*=1/(sum(relev1_array)+len(relev_array))
        else: 
            P1*=(relev1_array[palavra]+1)/(sum(relev1_array)+len(relev_array))
        if palavra not in relev2_array:
            P2*=1/(sum(relev2_array)+len(relev_array))
        else: 
            P2*= (relev2_array[palavra]+1)/(sum(relev2_array)+len(relev_array))
        if palavra not in relev3_array:
            P3*=1/(sum(relev3_array)+len(relev_array))
        else:
            P3*=(relev3_array[palavra]+1/(sum(relev3_array)+len(relev_array)))
        P1=P1*pr1
        P2=P2*pr2
        P3=P3*pr3
        x = [P1,P2,P3]
        if P1>P2 and P1>P3:
            return 1
        elif P2>P1 and P2>P3:
            return 2
        elif P3>P1 and P3>P2:
            return 3
    
    
    


Abaixo está presente um código utilizado por nós para testar a confiabilidade de nosso algoritmo. Logo, inserimos uma frase hipotética para ver como o código classifica o referido tweet.

In [23]:
#CRINDO UMA TABELA DE FREQUENCIA RELATIVA PARA UM TWEET ALEATORIO (FUTURO INPUT)
tweet_teste = 'era só um airpods na minha vida msm, sonho'
tweet_teste = tweet_teste.lower()
tweet_teste = cleanup(tweet_teste.lower())
tweet_teste = pd.Series(tweet_teste.split())
tweet_teste_relativo = tweet_teste.value_counts(True)

___
### Verificando a performance 1a Iteracao


Assim, criamos uma função classificador para testar a performance do algoritmo criado.
Logo, ao analisar o tweet, a função atribui um número (1,2 ou 3), referente ao grupo característico que classificou determinado tweet.
Tal número é atribuído na coluna classificador na linha do referido tweet.

In [18]:
#ABRINDO UM DATAFRAME E USANDO O CLASSIFICADOR (CRIANDO NOVO COLUNA DE CLASSIFICADOR NO DATAFRAME)

def classificador(df):
    df1 = pd.read_excel('{0}'.format(df))
    df1['Classificador']= 0
    cont = 0
    for tweet in df1['Teste']:
        tweet = cleanup(str(tweet))
        y = robo(tweet,1,1,1)
        df1.loc[cont,'Classificador']=y
        cont+=1
    return df1.loc[:,['Teste','Relevante.1','Classificador']]
    
        
    
    
    

O resultado está disponível abaixo.

In [36]:
#TESTANDO O CLASSIFICADOR COM O DATAFRAME DE TESTE
x = 'airpods1.xlsx'

teste =classificador(x)
#descomentar a linha abaixo para ver o resultado (dataframe com coluna de classificacao)
teste


,Teste,Relevante.1,Classificador
0,encontrei um que tem airpods em todas as fotos...,1.0,2
1,"era só um airpods na minha vida msm, sonho",3.0,3
2,o mta de nova york gostaria que você parasse d...,1.0,3
3,nao sei onde enfiei meus airpods e to tentando...,2.0,3
4,“shawn’s airpods #6” quantos airpods esse meni...,1.0,1
5,era só um airpods na minha vida mesmo,3.0,3
...,...,...,...
756,NaN,NaN,1
757,NaN,NaN,1
758,NaN,NaN,1


Por fim, fizemos uma função para verificar a performance do algoritmo feito.

Para tanto, adicionamos uma coluna com os resultados do algoritmo classificador ao nosso dataframe. Em seguida, criamos uma função que compara a coluna Teste, classificação feita por nós, manual, com a coluna Classificador, feita pelo algoritmo criado.

Para cada tweet que a classificação foi a mesma, é somado 1 ao contador. Ao fim da analise, é dividido o contador pelo total de tweets.
Como resultado, é apresentada uma porcentagem de acerto do algoritmo comparado com a nossa classificação, que serviu de base para a constituição do algoritmo.

Assim, a porcentagem de acerto (classificação igual do algoritmo com a nossa atribuição do tweet) foi de 40%.

In [38]:
#Calculadora de porcentagem de acertos
def calcula_acerto(teste):
    i = 0 
    contador = 0 
    while i < 249:
        if teste['Relevante.1'][i]== teste['Classificador'][i]:
            contador +=1
        i+=1
    porcentagem = (contador/248)*100
    porcentagem = round(porcentagem)
    return ('A porcentagem de acerto foi de {0}%'.format(porcentagem))
        

In [39]:
calcula_acerto(teste)

'A porcentagem de acerto foi de 40%'

## Alterando o metodo do classificador Naive Bayes

In [28]:
#Definindo interseccoes
#interseccao tweets relevantes=1 e todos os tweets 
set_relev1 = set(relev1_array.index)
set_relev = set(relev_array.index)
inter_R1_RT = set_relev1.intersection(set_relev)
lista1 = list(inter_R1_RT)
pr1 = len(lista1)/(len(relev_array)) #porcentagem  de palavras que aparecem na interseccao

#interseccao tweets relevantes=2 e todos os tweets 
set_relev2 = set(relev2_array.index)
set_relev = set(relev_array.index)
inter_R2_RT = set_relev2.intersection(set_relev)
lista2 = list(inter_R2_RT)
pr2 = len(lista2)/(len(relev_array)) #porcentagem  de palavras que aparecem na interseccao

#interseccao tweets relevantes=3 e todos os tweets 
set_relev3 = set(relev3_array.index)
set_relev = set(relev_array.index)
inter_R3_RT = set_relev3.intersection(set_relev)
lista3 = list(inter_R3_RT)
pr3 = len(lista3)/(len(relev_array)) #porcentagem  de palavras que aparecem na interseccao



In [29]:
#robo de probabilidade (agora usando a interseccao para calcular)
def robo_1(tweet_teste,P1,P2,P3,pr1,pr2,pr3):
    tweet_teste = str(tweet_teste)
    tweet_teste = tweet_teste.lower()
    tweet_teste = cleanup(tweet_teste)
    tweet_teste = removetext(tweet_teste)
    tweet_teste = pd.Series(tweet_teste.split())
    lista_tweet = list(tweet_teste)
    for palavra in lista_tweet:
        if palavra not in relev1_array or palavra not in relev2_array or palavra not in relev3_array:
            palavra = 1/(sum(relev1_array)+sum(relev2_array)+sum(relev3_array)+len(relev_array))
        else:
            P1*= (relev1_array[palavra]+1)/(sum(relev1_array)+len(relev_array))
            P2*= (relev2_array[palavra]+1)/(sum(relev2_array)+len(relev_array))
            P3*= (relev3_array[palavra]+1)/(sum(relev3_array)+len(relev_array))
        P1=P1*pr1
        P2=P2*pr2
        P3=P3*pr3
        x = [P1,P2,P3]
        if P1>P2 and P1>P3:
            return 1
        elif P2>P1 and P2>P3:
            return 2
        elif P3>P1 and P3>P2:
            return 3
    

In [30]:
#classificador 
def classificador_1(df,pr1,pr2,pr3):
    df1 = pd.read_excel('{0}'.format(df))
    df1['Classificador']= 0
    cont = 0
    for tweet in df1['Teste']:
        y = robo_1(tweet,1,1,1,pr1,pr2,pr3)
        df1.loc[cont,'Classificador']=y
        cont+=1
    return df1.loc[:248,['Teste','Relevante.1','Classificador']]

In [31]:
#rodando o primeiro classificador
x = 'airpods1.xlsx'
teste1= classificador_1(x,pr1,pr2,pr3)

## Verificando a performance da 2a Iteracao

In [32]:
#calculadora de porcentagem de acerto
def calcula_acerto_1(teste):
    i = 0
    contador = 0 
    while i < 249:
        if teste['Relevante.1'][i]== teste['Classificador'][i]:
            contador +=1
        i+=1
    porcentagem = (contador/248)*100
    porcentagem=round(porcentagem)
    return ('A porcentagem de acerto foi de {0}%'.format(porcentagem))
        

In [33]:
teste1

,Teste,Relevante.1,Classificador
0,encontrei um que tem airpods em todas as fotos...,1.0,1
1,"era só um airpods na minha vida msm, sonho",3.0,1
2,o mta de nova york gostaria que você parasse d...,1.0,1
3,nao sei onde enfiei meus airpods e to tentando...,2.0,1
4,“shawn’s airpods #6” quantos airpods esse meni...,1.0,1
5,era só um airpods na minha vida mesmo,3.0,1
...,...,...,...
243,hoje sonhei que o wani estragou os meus airpod...,1.0,1
244,não sabia dos airpods há mais de uma semana......,2.0,1
245,"nem queria comprar os airpods, mas agora que v...",3.0,1


In [34]:
w = calcula_acerto_1(teste1)
w

'A porcentagem de acerto foi de 54%'

In [35]:
pd.crosstab(teste1['Relevante.1'],teste1.Classificador,normalize = True)

Classificador,1,2,3
Relevante.1,,,
1.0,0.457831,0.008032,0.024096
2.0,0.208835,0.020080,0.012048
3.0,0.204819,0.008032,0.056225


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**